In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import load_img ,img_to_array , img_to_array


from sklearn.metrics import classification_report, confusion_matrix 
from mlxtend.plotting import plot_confusion_matrix


import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


import tensorflow.keras.utils as Utils
import keras
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from IPython.display import SVG
import seaborn as sns

import glob as gb

Using TensorFlow backend.


In [5]:

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 128,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

In [6]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [40]:
train_dataset_path = 'train'
test_image_path = "test_image"

In [8]:
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)


# train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, rescale=1./255,
#         shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')


train_generator = train_datagen.flow_from_directory(train_dataset_path, target_size=(128, 128),batch_size=batch_size,
        class_mode='binary')  

Found 1646 images belonging to 2 classes.


In [9]:
model.fit_generator(train_generator, steps_per_epoch=2000 // batch_size, epochs=3, validation_steps=800 // batch_size)


Epoch 1/3
125/125 [==============================] - 64s 511ms/step - loss: 0.6265 - accuracy: 0.6882
Epoch 2/3
125/125 [==============================] - 32s 257ms/step - loss: 0.5148 - accuracy: 0.7503
Epoch 3/3
125/125 [==============================] - 34s 273ms/step - loss: 0.4810 - accuracy: 0.7801


In [10]:
model.save_weights('emergency_or_not_new.h5') 

model.save("emergency_or_not_new.h5")

In [12]:

# img = load_img('1340.jpg')  # this is a PIL image
# x = img_to_array(img.resize([128,128]))  # this is a Numpy array with shape (3, 150, 150)
# x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)


# if (model.predict_classes(x)==1):
#     print ("Emergency")
# else:
#     print ("Non Emergency")
    
# img

In [58]:
def get_pred(directory):
    image_list = []
    
    image_name_list = []
    
    for image_file in os.listdir(directory): 
              
#         print("directory : ", directory , " image_file : ", image_file)

        image_name_list.append(image_file)
        
        image = cv2.imread(directory+ '/' +image_file) 
        image = cv2.resize(image,(128,128)) 
        image_list.append(image)
    
    return image_list , image_name_list

In [59]:
test_image , test_image_name_list = get_pred(test_image_path)
pred_Images = np.array(test_image)

print(pred_Images.shape)


pred_generator = ImageDataGenerator(rescale = 1/255)
pred_generator = pred_generator.flow(pred_Images, batch_size = 64, shuffle = False)

(706, 128, 128, 3)


In [82]:
# prediction = model.predict_generator(pred_generator, verbose=1)

prediction = model.predict_classes(pred_generator, verbose=1, batch_size = None)


12/12 [==============================] - 5s 443ms/step


In [85]:
def create_submission_file(model_prediction, filename = "default_prediction.csv"):
    
    submision = pd.read_csv("submission.csv")
    final_submission = submision.copy()
    
    final_submission["emergency_or_not"] = model_prediction

    final_submission.to_csv(filename , index = False)
    
    print(final_submission.head())
    
    return final_submission, filename + " file has successfully "
    
    

In [86]:
final_submission , message = create_submission_file(prediction, filename = "forth.csv")

  image_names  emergency_or_not
0    1960.jpg                 1
1     668.jpg                 0
2    2082.jpg                 1
3     808.jpg                 0
4    1907.jpg                 0


In [48]:
submision = pd.read_csv("submission.csv")
submision.head()

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,0
2,2082.jpg,0
3,808.jpg,0
4,1907.jpg,0


In [87]:
new_submission = submision.copy()

In [92]:
new_submission["image_test"] = test_image_name_list
new_submission["predicted_result"] = prediction

In [94]:
new_submission.head(5)

,image_names,emergency_or_not,image_test,predicted_result
0,1960.jpg,0,479.jpg,1
1,668.jpg,0,2265.jpg,0
2,2082.jpg,0,483.jpg,1
3,808.jpg,0,1472.jpg,0
4,1907.jpg,0,937.jpg,0


In [95]:
df11 = new_submission.copy()
df11.shape

(706, 4)

In [98]:
type(df11.image_names[0]), type(df11.image_test[0])

(str, str)